Step 4

In [1]:
import pandas as pd
import os

In [2]:
import neo4j

In [3]:
from neo4j import GraphDatabase

In [4]:
from graphdatascience import GraphDataScience

In [5]:
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")

In [6]:
NEO4J_AUTH = ( "neo4j",  "june15kent")

In [7]:
driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)

In [8]:
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

Step 5

In [9]:
pii_query = '''
MATCH (c:Client)-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]-> (n) <-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]-(d:Client)
WHERE id(c) < id(d)
RETURN c.id, d.id, count(*) AS freq
ORDER BY freq DESC
'''

In [10]:
gds.run_cypher(pii_query)

C:\Users\kentp\anaconda3\envs\cs350\Lib\site-packages\graphdatascience\query_runner\neo4j_query_runner.py:234: RuntimeWarning: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':HAS_EMAIL|HAS_PHONE|HAS_SSN' instead)
  warnings.warn(warning)


,c.id,d.id,freq
0,4952527271473904,4816336012071985,3
1,4883445100935916,4708373581412325,3
2,4658150168863397,4100374538108184,3
3,4673951123644611,4795773320377768,3
4,4192214340630620,4912097363222923,3
...,...,...,...
754,4910140986334626,4114683318919154,1
755,4454780847105236,4210575070378533,1
756,4721862020593706,4210575070378533,1
757,4445521165797820,4210575070378533,1


Step 6

In [11]:
project_query = '''
CALL gds.graph.project(
    'clientClusters' ,
    { Client: {
        label: 'Client' }
    },
    { SHARED_PII: {
        type: 'SHARED_PII',
        orientation: 'UNDIRECTED',
        properties: {
        count: {
        property: 'count' }
            }
        }
    }
)
YIELD graphName, nodeCount, relationshipCount
'''

In [22]:
gds.run_cypher(project_query)

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: A graph with name 'clientClusters' already exists.}

Step 7

In [13]:
streaming_query = '''
CALL gds.wcc.stream(
    'clientClusters',
    {
    nodeLabels: ['Client'],
    relationshipTypes: ['SHARED_PII'],
    consecutiveIds: true
    }
)
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).id AS clientId, 
componentId AS clusterId
'''

In [14]:
pandasDF = driver.execute_query(
    streaming_query,
    database_="neo4j",
    result_transformer_=  neo4j.Result.to_df
)

In [15]:
print(type(pandasDF))

<class 'pandas.core.frame.DataFrame'>


In [16]:
with driver.session() as session:
    result = session.run(streaming_query)
    print(result.single())

<Record clientId='4997933060327094' clusterId=0>


C:\Users\kentp\anaconda3\envs\cs350\Lib\site-packages\neo4j\_sync\work\result.py:620: UserWarning: Expected a result with a single record, but found multiple.
  warn(


Clean Up

In [17]:
gds.run_cypher('''CALL gds.graph.drop('nativeGraph', false)''')

,graphName,database,databaseLocation,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema,schemaWithOrientation


Step 9

In [18]:
bipartite_graph = '''
MATCH (c:Client) WHERE c.secondPartyFraudRing is NOT NULL
WITH collect(c) as clients

MATCH (n) WHERE n:Email OR n:Phone OR n:SSN

WITH clients, collect(n) AS piis
WITH clients + piis AS nodes

MATCH (c:Client) -[:HAS_EMAIL | :HAS_PHONE | :HAS_SSN]->(p)
WHERE c.secondPartyFraudRing is NOT NULL

WITH nodes, collect({source: c, target: p}) as relationships

CALL gds.graph.project.cypher(
    'similarity',
    "UNWIND $nodes as n
        RETURN id(n) AS id,labels(n) AS labels",
    "UNWIND $relationships as r
        RETURN id(r['source']) AS source, id(r['target']) AS target,
    'HAS_PII' as type",
    { parameters:
        { nodes: nodes,
            relationships: relationships }
    }
)
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount

'''

In [19]:
gds.run_cypher(bipartite_graph)

C:\Users\kentp\anaconda3\envs\cs350\Lib\site-packages\graphdatascience\query_runner\neo4j_query_runner.py:234: RuntimeWarning: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':HAS_EMAIL|HAS_PHONE|HAS_SSN' instead)
  warnings.warn(warning)
C:\Users\kentp\anaconda3\envs\cs350\Lib\site-packages\graphdatascience\query_runner\neo4j_query_runner.py:234: RuntimeWarning: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: secondPartyFraudRing)
  warnings.warn(warning)


,graphName,nodeCount,relationshipCount


Step 10

In [20]:
nodeSimilarity_mutate = '''

CALL gds.nodeSimilarity.mutate(
  'FraudDetectionAssignment',
  {
    mutateProperty: 'jaccardScore',
    mutateRelationshipType: 'SIMILAR_TO',
    topK: 15
  }
)
YIELD
  preProcessingMillis,
  computeMillis,
  mutateMillis,
  postProcessingMillis,
  relationshipsWritten,
  nodesCompared,
  similarityDistribution,
  configuration
'''

In [21]:
gds.run_cypher(nodeSimilarity_mutate)

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.nodeSimilarity.mutate`: Caused by: java.util.NoSuchElementException: Graph with name `FraudDetectionAssignment` does not exist on database `neo4j`. It might exist on another database.}